# Preprocess into spacy's Doc/DocBin format

In [ ]:
import gzip

import pandas as pd
import spacy

In [ ]:
with gzip.open("../input/nbme-score-clinical-patient-notes/train_split.json.gz", "r") as fin:
    df = pd.read_json(fin)

In [ ]:
df.head()

## Select one example case and feature number

In [ ]:
case_num = 0
feature_num = 0
sub_df = df.loc[(df.case_num == case_num) & (df.feature_num == feature_num), :]

In [ ]:
sub_df.head()

In [ ]:
sub_df.fold.value_counts()

In [ ]:
feature_text = sub_df.feature_text.unique()
assert len(feature_text) == 1
feature_text[0]

### Validate and visualize spans

In [ ]:
sub_df.location
# TODO clean this

In [ ]:
start_pos = []
end_pos = []
for loc_list in patient_df.location:
    for loc in loc_list:
        start_pos.append(loc[0])
        end_pos.append(loc[1])
        
ents = []
for start, end in zip(start_pos, end_pos):
    ents.append({
        'start': start, 
        'end' : end,
        'label' : 'L'
    })
doc = {
    'text' : patient_notes[patient_notes["pn_num"] == PATIENT_IDX]["pn_history"].iloc[0],
    "ents" : ents
}
colors = {"L" :"linear-gradient(90deg, #aa9cfc, #fc9ce7)" } 
options = {"colors": colors}
spacy.displacy.render(doc, style="ent", options = options , manual=True, jupyter=True);